In [1]:
!pip install sktime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 58.0 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels
#import xgboost as xgb

In [3]:
Y_data = pd.read_csv(r"https://raw.githubusercontent.com/ladyjossy77/hierarchical-forecasting/master/data/forecast.csv", index_col=0)

In [4]:
Y_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9240 entries, 0 to 9239
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   CustomerContinent  9240 non-null   object 
 1   ProductCategory    9240 non-null   object 
 2   dateOfPurchase     9240 non-null   object 
 3   TotalQuantity      6611 non-null   float64
 4   TotalRevenue       6611 non-null   float64
dtypes: float64(2), object(3)
memory usage: 433.1+ KB


In [5]:
Y_data["dateOfPurchase"] = pd.to_datetime(Y_data["dateOfPurchase"]).dt.to_period('M')

In [6]:
Y_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9240 entries, 0 to 9239
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype    
---  ------             --------------  -----    
 0   CustomerContinent  9240 non-null   object   
 1   ProductCategory    9240 non-null   object   
 2   dateOfPurchase     9240 non-null   period[M]
 3   TotalQuantity      6611 non-null   float64  
 4   TotalRevenue       6611 non-null   float64  
dtypes: float64(2), object(2), period[M](1)
memory usage: 433.1+ KB


In [7]:
Y_data = Y_data.groupby(["CustomerContinent", "ProductCategory", "dateOfPurchase"]).agg(
                                TotalQuantity = pd.NamedAgg(column = "TotalQuantity", aggfunc=sum),
                                TotalRevenue = pd.NamedAgg(column = "TotalRevenue", aggfunc = sum)
)

In [8]:
y_train = Y_data[Y_data.index.get_level_values('dateOfPurchase')<"2019-01"]
y_validate = Y_data[Y_data.index.get_level_values('dateOfPurchase')>="2019-01"]

In [9]:
#standardizing data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
y_train.values
index = y_train.index
columns = y_train.columns
scaled_values = scaler.fit_transform(y_train.values)

In [10]:
scaled_values

array([[-0.87405298, -0.68006979],
       [ 1.40614653,  0.50069846],
       [ 0.10317539,  0.47950609],
       ...,
       [-0.65689112, -0.33744483],
       [ 1.62330839,  2.06909531],
       [-0.11398647,  0.09687404]])

In [11]:
scaled_y = pd.DataFrame(scaled_values, columns = columns, index = index)
scaled_y.head()

TotalQuantity  TotalRevenue
CustomerContinent ProductCategory dateOfPurchase                             
Africa            Automotive      2016-01             -0.874053     -0.680070
                                  2016-02              1.406147      0.500698
                                  2016-03              0.103175      0.479506
                                  2016-04              1.514727      1.294496
                                  2016-05              0.971823      0.794730

In [12]:
from sktime.forecasting.base import ForecastingHorizon
fh = ForecastingHorizon(
    pd.PeriodIndex(pd.date_range("2019-01", periods= 24, freq="M")), is_relative=False
)
fh

ForecastingHorizon(['2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06',
             '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
             '2020-01', '2020-02', '2020-03', '2020-04', '2020-05', '2020-06',
             '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12'],
            dtype='period[M]', is_relative=False)

In [13]:
from sktime.forecasting.var import VAR
from sktime.forecasting.varmax import VARMAX
from sktime.forecasting.dynamic_factor import DynamicFactor
from sktime.forecasting.model_selection._tune import ForecastingGridSearchCV
from sktime.forecasting.compose._pipeline import Permute
from sktime.forecasting.compose._pipeline import TransformedTargetForecaster
from sktime.forecasting.compose._hierarchy_ensemble import HierarchyEnsembleForecaster
from sktime.forecasting.compose._ensemble import EnsembleForecaster
import statsmodels

forecaster_set = [("var", VAR()), 
                  ("varmax", VARMAX())]


To improve the performance of Ensemble Forecaster with VAR and VARMAX as base estimators, we will tune hyperparamters with GrisdSearch CV

In [14]:


from sktime.forecasting.model_selection import SlidingWindowSplitter
from sktime.forecasting.model_selection import SingleWindowSplitter
forecaster_set = [("var", VAR()), 
                  ("varmax", VARMAX())]
forecaster = EnsembleForecaster(forecasters= forecaster_set, weights= [4,10])
# fh = ForecastingHorizon(
#     pd.PeriodIndex(pd.date_range("2019-01", periods= 24, freq="M")), is_relative=False
# )
fh = pd.PeriodIndex(pd.date_range("2019-01", periods= 24, freq="M"))
horizon=pd.Series({"CustomerContinent": fh, "ProductCategory": fh})
# cv = SlidingWindowSplitter(fh=horizon, window_length= 24)
cv = SingleWindowSplitter(fh = 24)


In [15]:
EnsembleForecaster(forecaster_set).get_params().keys()

dict_keys(['aggfunc', 'forecasters', 'n_jobs', 'weights', 'var', 'varmax', 'var__dates', 'var__freq', 'var__ic', 'var__maxlags', 'var__method', 'var__missing', 'var__random_state', 'var__trend', 'var__verbose', 'varmax__callback', 'varmax__cov_kwds', 'varmax__cov_type', 'varmax__disp', 'varmax__dynamic', 'varmax__enforce_invertibility', 'varmax__enforce_stationarity', 'varmax__error_cov_type', 'varmax__flags', 'varmax__full_output', 'varmax__includes_fixed', 'varmax__information_set', 'varmax__low_memory', 'varmax__maxiter', 'varmax__measurement_error', 'varmax__method', 'varmax__optim_complex_step', 'varmax__optim_hessian', 'varmax__optim_score', 'varmax__order', 'varmax__return_params', 'varmax__signal_only', 'varmax__start_params', 'varmax__suppress_warnings', 'varmax__transformed', 'varmax__trend', 'varmax__trend_offset'])

In [16]:
param_grid = {
    'var__ic': ['aic', 'fpe', 'hqic', 'bic', None],
    'var__method' :["ols","gls", "gmm","ml", "irf","mvd"],
    'var__trend':["c", "ct", "ctt", "n"],
    'varmax__method':["newton","nf","bfgs","lbfgs", "powell","cg","ncg","basinhopping"],
    'varmax__trend':["n", "c","t", "ct"],
    'weights': [(0.33, 0.67), (0.6, 0.4), (0.5,0.5)],
    'n_jobs':[-1],
    'aggfunc':['mean', 'median', 'min', 'max']
}

In [17]:
from sktime.performance_metrics.forecasting import mean_absolute_percentage_error
gscv = ForecastingGridSearchCV(forecaster= forecaster, cv=cv, scoring= mean_absolute_percentage_error, param_grid= param_grid)

In [18]:
horizon

CustomerContinent    PeriodIndex(['2019-01', '2019-02', '2019-03', ...
ProductCategory      PeriodIndex(['2019-01', '2019-02', '2019-03', ...
dtype: object

In [19]:
#model failed to fit to E-F, will resort to random search with feweer params
# tuned_model_one = gscv.fit(y=scaled_y, fh= fh)


In [21]:
y_train.isna().sum()

TotalQuantity    0
TotalRevenue     0
dtype: int64

In [22]:
from sktime.forecasting.model_selection._tune import ForecastingRandomizedSearchCV

In [23]:
param_grid = {
    'var__ic': ['aic', 'fpe', 'hqic'],
    'var__method' :["ols","gls", "gmm"],
    'var__trend':["c", "ct"],
    'varmax__method':["newton","nf","bfgs"],
    'varmax__trend':["n", "c","t", ],
    'weights': [(0.33, 0.67), (0.6, 0.4), (0.5,0.5)],
    'n_jobs':[-1],
    'aggfunc':['mean', 'median']
}

In [25]:
rdscv = ForecastingRandomizedSearchCV(forecaster= forecaster, cv=cv, scoring= mean_absolute_percentage_error, param_distributions= param_grid)

In [27]:
#model failed fitting

# tuned_model_two = rdscv.fit(y=scaled_y, fh= fh)

In [29]:
#reducing fh
fh  = pd.PeriodIndex(pd.date_range("2019-01", periods= 6, freq="M"))

In [31]:
# tuned_model_three = rdscv.fit(y=scaled_y, fh= fh)

#all fits failed even after using a shorter fh

Using other base estimators to train model

In [33]:
from sktime.forecasting.dynamic_factor import DynamicFactor
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.transformations.series.impute import Imputer
from sktime.transformations.series.detrend import Detrender

In [34]:
TTF = TransformedTargetForecaster(steps=[
    ("imputer", Imputer(method="mean")),
    ("detrender", Detrender()),
    ("forecaster", VAR()),
])

In [36]:
forecasters = [("dynamic", DynamicFactor()),
               ("transformer", TTF)]

In [37]:
forecaster = EnsembleForecaster(forecasters= forecasters)

In [38]:
tuned_model_four = forecaster.fit(y = scaled_y, fh = fh)

In [39]:
predict_tuned_four = tuned_model_four.predict()
predict_tuned_four.head()

TotalQuantity  TotalRevenue
CustomerContinent ProductCategory dateOfPurchase                             
Africa            Automotive      2019-01              0.253502      0.350055
                                  2019-02              0.212094      0.288692
                                  2019-03              0.222176      0.311205
                                  2019-04              0.219296      0.315622
                                  2019-05              0.218654      0.323363

In [40]:
predicted_index = predict_tuned_four.index
predicted_columns = predict_tuned_four.columns

inversed_prediction = scaler.inverse_transform(predict_tuned_four)
inversed_prediction
predicted_data = pd.DataFrame(inversed_prediction, index = predicted_index, columns = predicted_columns)
predicted_data.head()



TotalQuantity  TotalRevenue
CustomerContinent ProductCategory dateOfPurchase                             
Africa            Automotive      2019-01             11.384469   6000.005516
                                  2019-02             11.003111   5699.854354
                                  2019-03             11.095959   5809.973193
                                  2019-04             11.069435   5831.581188
                                  2019-05             11.063525   5869.446501

In [43]:
y_test = y_validate[y_validate.index.get_level_values('dateOfPurchase')<"2019-07"]

In [44]:
y_test.tail()

TotalQuantity  TotalRevenue
CustomerContinent ProductCategory dateOfPurchase                             
South America     Toys            2019-02                  24.0      14165.73
                                  2019-03                   5.0       1651.30
                                  2019-04                   5.0       1205.75
                                  2019-05                   0.0          0.00
                                  2019-06                   7.0       6278.58

In [45]:
predicted_data.tail()

TotalQuantity  TotalRevenue
CustomerContinent ProductCategory dateOfPurchase                             
South America     Toys            2019-02              6.949002   3607.987326
                                  2019-03              7.001069   3645.563807
                                  2019-04              7.027273   3670.478193
                                  2019-05              7.046126   3692.006322
                                  2019-06              7.062783   3712.541021

In [47]:
from sklearn.metrics import r2_score

In [48]:
score_tuned_four = r2_score(y_test, predicted_data)
score_tuned_four

0.1848163974433557

Generate exogenous data to improve model performance